# READY TO BE DISCHARGED: EXAMINING HOSPITAL READMISSIONS

passar para x_ as categoricas para aplicar um feature selection

<div class="alert alert-block alert-info">

[1. Objectives](#1st-bullet)<br>
[2. Import Data/Libraries](#2nd-bullet)<br>
- [2.1 Import Libraries](#3rd-bullet)<br>
- [2.2 Import Data](#4th-bullet)<br>    
 
[3. Data Exploration](#5th-bullet)<br>
- [3.1 Missing Values](#6th-bullet)<br>
- [3.2 Detailed Look into Variables](#7th-bullet)<br> 
    - [3.2.1 Metric Variables](#8th-bullet)<br>
    - [3.2.2 Non-Metric Variables](#9th-bullet)<br>
- [3.3 Duplicates](#10th-bullet)<br> 

</div>

<div class="alert alert-block alert-success">

<a class="anchor" id="1st-bullet">    </a>
## 1. Objectives
    

    
</div>


- **Binary Classification:** predict if a patient will be readmitted to the hospital within 30 days of being discharged. 
- **Multiclass Classification:** predict the timeframe of a patient’s readmission, with the classes being “No”, “<30 days”, “>30 days”.

<div class="alert alert-block alert-success">

<a class="anchor" id="2nd-bullet">    </a>
## 2. Import Data/Libraries
    

    
</div>

<a class="anchor" id="3rd-bullet">

### 2.1. Import Libraries
    
</a>

In [133]:
import pandas as pd
import numpy as np

In [134]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from math import ceil
import os
import scipy.stats as stats

#data preparation
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split

#model importation
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

#feature selection
from sklearn.feature_selection import RFE, RFECV
from sklearn.linear_model import LassoCV, RidgeCV

from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score


#ignore warnings
import warnings
warnings.filterwarnings('ignore') 

<a class="anchor" id="4th-bullet">

### 2.2. Import Data
    
</a>

In [135]:
data= pd.read_csv('train.csv') 

<div class="alert alert-block alert-success">

<a class="anchor" id="5th-bullet">    </a>
## 3. Data Exploration
    

    
</div>

In [136]:
data.replace("?", np.nan, inplace=True)

In [137]:
data = data.drop(['country','weight'], axis = 1)

In [138]:
data.set_index("encounter_id",inplace=True)

In [139]:
data.drop('patient_id', axis =1, inplace = True)

In [140]:
data['hospital_visits_previous_year'] = data['outpatient_visits_in_previous_year'] + data['inpatient_visits_in_previous_year'] +data['emergency_visits_in_previous_year']
#data.drop(['outpatient_visits_in_previous_year','inpatient_visits_in_previous_year','emergency_visits_in_previous_year'], axis =1, inplace = True)

In [141]:
data['x_high_bpm'] = data['average_pulse_bpm'].map(lambda g: 1 if g>85 else 0)
data.drop('average_pulse_bpm', axis =1, inplace = True)

In [142]:
#non_metric_features

In [143]:
data['x_race_minority'] = data['race'].map(lambda g: 0 if g == 'Caucasian' else 1).astype(object)
data.drop('race', axis =1, inplace = True)

In [144]:
data = data[data['gender'] != 'Unknown/Invalid']

In [145]:
data["x_gender"]=data["gender"].map(lambda g: 1 if g=="Male" else 0).astype(object)
data.drop('gender', axis =1, inplace = True)

In [146]:
def age_conversion(age):
    if age is np.nan:
        return np.nan
    else:
        return age[1]

In [147]:
data["x_age"]=data["age"].map(age_conversion).astype(object)
data.drop('age', axis =1, inplace = True)

In [148]:
data['x_medicare'] = data['payer_code'].map(lambda g: 1 if g =='MC' else 0 ).astype(object)

In [149]:
data['x_insurance'] = data['payer_code'].map(lambda g: 0 if g in ['MC', 'HM', 'UN', 'SP', 'SI', 'CM', 'DM', 'CP', 'MD', 'OG',
       'BC', 'PO', 'WC', 'OT', 'MP', 'CH', 'FR'] else 1 ).astype(object)
data.drop('payer_code', axis = 1, inplace = True)

In [150]:
dict_adm = {'Emergency': 1, 'Elective': 3, 'Urgent': 2, 'Not Available':0, 'Not Mapped':0, 'Trauma Center': 5, 'Newborn':4} 
data.replace({"admission_type": dict_adm},inplace=True)

data['x_admission_type'] = data['admission_type'].astype(object)
data.drop('admission_type', axis =1, inplace = True)

In [151]:
data['medical_specialty'].replace("'PhysicianNotFound'", np.nan, inplace=True)

In [152]:
def medical_specialty_group(medical_specialty):
    evaluation_managment = {'Family/GeneralPractice','InternalMedicine','Emergency/Trauma','Hospitalist','DCPTEAM','Resident'}
    surgery ={'Anesthesiology-Pediatric','Anesthesiology','Surgery-Pediatric','Surgery-Colon&Rectal','Surgeon','SurgicalSpecialty','Surgery-General','Surgery-Plastic','Surgery-Cardiovascular','Podiatry','Surgery-Maxillofacial','Surgery-Thoracic','Orthopedics-Reconstructive','Surgery-Vascular','Surgery-Neuro','Surgery-Cardiovascular/Thoracic','Obstetrics'}
    patho_lab_radio = {'Hematology','Hematology/Oncology','Pathology','Pediatrics-Hematology-Oncology','Radiologist','Oncology','Radiology','Obsterics&Gynecology-GynecologicOnco'}
    medicine = {'Neurophysiology','Proctology','Pediatrics-AllergyandImmunology','Endocrinology-Metabolism', 'OutreachServices','Speech','Dentistry','Psychiatry-Child/Adolescent','Cardiology-Pediatric','Pediatrics-EmergencyMedicine','Rheumatology','Nephrology','Osteopath','Pediatrics-Pulmonology','AllergyandImmunology','InfectiousDiseases','Pediatrics-Neurology','Otolaryngology','Ophthalmology','Endocrinology','Orthopedics','Psychiatry','Cardiology','ObstetricsandGynecology','Gastroenterology','Psychology','Neurology','PhysicalMedicineandRehabilitation','Pulmonology','Urology','Pediatrics','Gynecology','Pediatrics-Endocrinology','Pediatrics-CriticalCare'}
    if medical_specialty in evaluation_managment:
        return 1 #'Evaluation'
    if medical_specialty in surgery:
        return 2 #'Surgery'
    if medical_specialty in patho_lab_radio:
        return 3 #'Patholody/Radiology'
    if medical_specialty in medicine:
        return 0 # 'Medicine'
    return np.nan

In [153]:
data["x_medical_specialty"]=data["medical_specialty"].map(medical_specialty_group).astype(object)
data.drop('medical_specialty', axis =1, inplace = True)

In [154]:
def care_after_discharge_conversion(discharge_disposition):
    snf={"Discharged/transferred to SNF",
        "Discharged/transferred to a nursing facility certified under Medicaid but not certified under Medicare."}
    home={"Discharged/transferred to home with home health service",
        "Discharged/transferred to home under care of Home IV provider",
        "Discharged to home"}
    rehab={"Discharged/transferred to another rehab fac including rehab units of a hospital ."}
    short={"Discharged/transferred to another short term hospital",
        "Discharged/transferred to another type of inpatient care institution",
        "Discharged/transferred/referred to this institution for outpatient services",
        "Discharged/transferred/referred another institution for outpatient services",
        "Still patient or expected to return for outpatient services"}
    icf={"Discharged/transferred to ICF"}
    hospice={"Hospice / medical facility","Hospice / home",
        "Expired at home. Medicaid only, hospice.",
        "Discharged/transferred to a federal health care facility."}
    long={"Discharged/transferred to a long term care hospital.",
        "Discharged/transferred within this institution to Medicare approved swing bed",
        "Admitted as an inpatient to this hospital",
        "Neonate discharged to another hospital for neonatal aftercare"}
    psychiatry={"Discharged/transferred/referred to a psychiatric hospital of psychiatric distinct part unit of a hospital"}
    left={"Left AMA"}
    if discharge_disposition in snf:
        return "snf"
    if discharge_disposition in home:
        return "home"
    if discharge_disposition in rehab:
        return "rehab"
    if discharge_disposition in short:
        return "short"
    if discharge_disposition in icf:
        return "icf"
    if discharge_disposition in hospice:
        return "hospice"
    if discharge_disposition in long:
        return "long"
    if discharge_disposition in psychiatry:
        return "psychiatry"
    if discharge_disposition in left:
        return "left"
    return "other"

In [155]:
data["discharge_disposition"]=data["discharge_disposition"].\
    map(care_after_discharge_conversion).astype("category")
for cat in ["snf","home","rehab","short","icf","hospice","long","psychiatry","left"]:
    data["x_discharge_("+cat+")"]=(data["discharge_disposition"]==cat).astype(int).astype(object)
data.drop("discharge_disposition",axis=1,inplace=True)

In [156]:
def admission_source_conversion(admission_source):
    emergency={" Emergency Room"}
    transfer={"Transfer from a hospital",
        " Transfer from another health care facility",
        " Transfer from a Skilled Nursing Facility (SNF)",
        " Transfer from hospital inpt/same fac reslt in a sep claim",
        " Transfer from critial access hospital",
        " Transfer from Ambulatory Surgery Center"}
    referral={" Physician Referral","Clinic Referral","HMO Referral"}
    if admission_source in emergency:
        return  1 #"Emergency"
    if admission_source in transfer:
        return 2 #"Transfer"
    if admission_source in referral:
        return 3 #"Referral"
    return np.nan

In [157]:
data["x_admission_source"]=data["admission_source"].map(admission_source_conversion).astype(object)
data.drop('admission_source', axis =1, inplace = True)

In [158]:
data['primary_diagnosis'] = data['primary_diagnosis'].astype(object)

In [159]:
def is_diabetes(code):
    if  '.' in code or code == '250':
        return 1
    else:
        return 0

In [160]:
data["diab_prim"]=(data["primary_diagnosis"].astype(str).map(is_diabetes)).astype(object)
data["diab_sec"]=(data["secondary_diagnosis"].astype(str).map(is_diabetes)).astype(object)
data["diab_add"]=(data["additional_diagnosis"].astype(str).map(is_diabetes)).astype(object)
data.drop(["primary_diagnosis","secondary_diagnosis" ,"additional_diagnosis"], axis =1 , inplace = True)

In [161]:
data['diabetes_diag'] = data["diab_prim"]+data["diab_sec"]+data["diab_add"]
data['x_diabetes_diag'] = data['diabetes_diag'].map(lambda g: 1 if g>0 else 0).astype(object)
data.drop(["diab_prim","diab_sec" ,"diab_add", 'diabetes_diag'], axis =1 , inplace = True)

In [162]:
data['x_glucose_diabetes'] = data['glucose_test_result'].map(lambda g: 0 if g in ['Norm', np.nan] else 1).astype(object)
data.drop('glucose_test_result', axis =1 , inplace = True)

In [163]:
data['x_a1c_diabetes'] = data['a1c_test_result'].map(lambda g: 0 if g in ['Norm', np.nan] else 1).astype(object)
data.drop('a1c_test_result', axis =1 , inplace = True)

In [164]:
data['x_change_in_meds_during_hospitalization'] = data['change_in_meds_during_hospitalization'].map(lambda g: 0 if g== 'No' else 1).astype(object)
data.drop('change_in_meds_during_hospitalization', axis =1, inplace = True)

In [165]:
data['x_prescribed_diabetes_meds'] = data['prescribed_diabetes_meds'].map(lambda g: 0 if g== 'No' else 1).astype(object)
data.drop('prescribed_diabetes_meds', axis =1, inplace = True)

In [166]:
def insulin_flagger (medicine):
    if 'insulin' in medicine:
        return 1
    return 0

In [167]:
def metformin_flagger (medicine):
    if 'metformin' in medicine:
        return 1
    return 0

In [168]:
def count_medication(med):
    count=0
    for char in med:
        if char=="'":
            count+=1
    return count//2

In [169]:
data["#_of_diab_meds"]=data["medication"].map(count_medication).astype(object)

In [170]:
data['x_insulin_medication'] = data['medication']. map(insulin_flagger).astype(object)
data['x_metformin_medication'] = data['medication']. map(metformin_flagger).astype(object)
data.drop('medication', axis = 1, inplace= True)

In [171]:
#data['hospital_work'] = data['length_of_stay_in_hospital'] + data['number_lab_tests'] + data['non_lab_procedures']+ data['hospital_visits_previous_year']+ data['#_of_diab_meds']

# Dealing with MV and Outliers:

In [172]:
df_1 = data.copy()

## Missing values:

In [173]:
non_metric_features = df_1.dtypes[df_1.dtypes == object].index.tolist()
metric_features = df_1.columns.drop(non_metric_features).to_list()

In [174]:
modes = df_1[non_metric_features].mode().loc[0]
df_1[non_metric_features] = df_1[non_metric_features].fillna(modes).astype(object)

## Separate Target:

In [175]:
X_2 = df_1.drop(['readmitted_binary', 'readmitted_multiclass'], axis = 1)

#y_binary and y_multiclass are encoded so they work for all methods:
y_binary_2 = df_1['readmitted_binary'].replace(['No', 'Yes'], [0, 1])

In [176]:
X = data.drop(['readmitted_binary', 'readmitted_multiclass'], axis = 1)

#y_binary and y_multiclass are encoded so they work for all methods:
y_binary = data['readmitted_binary'].replace(['No', 'Yes'], [0, 1])

# Modeling:

In [177]:
import warnings
warnings.filterwarnings('ignore')
import time
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, RobustScaler
import numpy as np
from sklearn.naive_bayes import MultinomialNB, GaussianNB, ComplementNB, BernoulliNB, CategoricalNB
from sklearn.ensemble import  HistGradientBoostingClassifier
#set random seed
np.random.seed(0)

In [178]:
categorical = [0,1,5,6,7,13,14,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34]

In [179]:
lr = LogisticRegression(class_weight={0: 0.11, 1: 0.89},
                        solver = 'newton-cg',
                       
                        )

In [180]:
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier(class_weight= {0: 0.11, 1: 0.89}
                    ,loss ='log_loss'
                    ,penalty = 'elasticnet'
                    ,learning_rate= 'adaptive'
                    ,eta0 = 0.1
                    ,validation_fraction = 0.2
                    ,random_state=7
                    )

In [181]:
hgb = HistGradientBoostingClassifier(class_weight={0:0.14,
                                                     1:0.86},
                                       learning_rate=0.05,
                                       random_state=7,
                                       max_iter= 60,
                                       max_leaf_nodes= 40)

In [182]:
from sklearn.ensemble import RandomForestClassifier

In [183]:
rf = RandomForestClassifier(class_weight={0:0.12,
                                          1:0.88},
                             n_estimators= 100,
                             max_depth= 100,
                             min_samples_split= 0.02,
                            random_state = 46
                            
                            )

In [184]:
nn = MLPClassifier(activation = 'logistic', hidden_layer_sizes=(8), max_iter = 1000, random_state=42)

In [185]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB, ComplementNB, BernoulliNB, CategoricalNB
nb = GaussianNB()

In [186]:
def avg_score2(model, scale): 
    
    # apply kfold
    skf = StratifiedKFold(n_splits=10)
    # create lists to store the results from the different models 
    score_train = []
    score_test = []
    timer = []
    n_iter = []
    for train_index, test_index in skf.split(X_2,y_binary):
        # get the indexes of the observations assigned for each partition
        X_train, X_val = X_2.iloc[train_index], X_2.iloc[test_index]
        y_train, y_val = y_binary.iloc[train_index], y_binary.iloc[test_index]
        
        if scale == 'minmax':
            #scale your data using MinMaxScaler[-1,1]
            # Create a MinMaxScaler instance that will range between -1 and 1 and fit to your train data
            min_max = MinMaxScaler().fit(X_train)
            # Transform your train data by applying the scale obtained in the previous command
            scaled_X_train = min_max.transform(X_train)
            # Transform your validation data by applying the scale obtained in the first command
            scaled_X_val = min_max.transform(X_val)
        elif scale == 'minmax2':
            min_max = MinMaxScaler(feature_range=(-1, 1)).fit(X_train)
            # Transform your train data by applying the scale obtained in the previous command
            scaled_X_train = min_max.transform(X_train)
            # Transform your validation data by applying the scale obtained in the first command
            scaled_X_val = min_max.transform(X_val)
        elif scale == 'robust':
            robust = RobustScaler().fit(X_train)
            # Transform your train data by applying the scale obtained in the previous command
            scaled_X_train = robust.transform(X_train)
            # Transform your validation data by applying the scale obtained in the first command
            scaled_X_val = robust.transform(X_val)
        
        # start counting time
        begin = time.perf_counter()
        # fit the model to the data
        model.fit(scaled_X_train, y_train)
        predict_train = model.predict(X_train)
        predict_val = model.predict(X_val)
        # finish counting time
        end = time.perf_counter()
        # check the mean accuracy for the train
        value_train = f1_score(y_train, predict_train)
        # check the mean accuracy for the test
        value_test = f1_score(y_val, predict_val)
        # append the accuracies, the time and the number of iterations in the corresponding list
        score_train.append(value_train)
        score_test.append(value_test)
        timer.append(end-begin)
    # calculate the average and the std for each measure (accuracy, time and number of iterations)
    avg_time = round(np.mean(timer),3)
    avg_train = round(np.mean(score_train),3)
    avg_test = round(np.mean(score_test),3)
    std_time = round(np.std(timer),2)
    std_train = round(np.std(score_train),2)
    std_test = round(np.std(score_test),2)
    
    return str(avg_time) + '+/-' + str(std_time), str(avg_train) + '+/-' + str(std_train),\
str(avg_test) + '+/-' + str(std_test)

def show_results2(df, scale, *args):
    """
    Receive an empty dataframe and the different models and call the function avg_score
    """
    count = 0
    # for each model passed as argument
    scale = scale
    for arg in args:
        # obtain the results provided by avg_score
        time, avg_train, avg_test = avg_score2(arg, scale)
        # store the results in the right row
        df.iloc[count] = time, avg_train, avg_test
        count+=1
    return df

In [187]:
def avg_score_ns(model):
    # apply kfold
    skf = StratifiedKFold(n_splits=10)
    # create lists to store the results from the different models 
    score_train = []
    score_test = []
    timer = []
    
    #encoding too
    
    for train_index, test_index in skf.split(X_2,y_binary_2):
        # get the indexes of the observations assigned for each partition
        X_train, X_val = X_2.iloc[train_index], X_2.iloc[test_index]
        y_train, y_val = y_binary_2.iloc[train_index], y_binary_2.iloc[test_index]
    
       
        # start counting time
        begin = time.perf_counter()
        # fit the model to the data
        model.fit(X_train, y_train)
        predict_train = model.predict(X_train)
        predict_val = model.predict(X_val)
        # finish counting time
        end = time.perf_counter()
        # check the mean accuracy for the train
        value_train = f1_score(y_train, predict_train)
        # check the mean accuracy for the test
        value_test = f1_score(y_val, predict_val)
        # append the accuracies, the time and the number of iterations in the corresponding list
        score_train.append(value_train)
        score_test.append(value_test)
        timer.append(end-begin)
        
    # calculate the average and the std for each measure (accuracy, time and number of iterations)
    avg_time = round(np.mean(timer),3)
    avg_train = round(np.mean(score_train),3)
    avg_test = round(np.mean(score_test),3)
    std_time = round(np.std(timer),2)
    std_train = round(np.std(score_train),2)
    std_test = round(np.std(score_test),2)
    
    return str(avg_time) + '+/-' + str(std_time), str(avg_train) + '+/-' + str(std_train),\
str(avg_test) + '+/-' + str(std_test)

def show_results_ns(df, *args):
    """
    Receive an empty dataframe and the different models and call the function avg_score
    """
    count = 0
    # for each model passed as argument
    #scale = scale
    for arg in args:
        # obtain the results provided by avg_score
        time, avg_train, avg_test = avg_score_ns(arg)
        # store the results in the right row
        df.iloc[count] = time, avg_train, avg_test
        count+=1
    return df

In [188]:
def avg_score_ns_mv(model):
    # apply kfold
    skf = StratifiedKFold(n_splits=10)
    # create lists to store the results from the different models 
    score_train = []
    score_test = []
    timer = []
    acc_test = []
    acc_train = []
    
    #encoding too
    
    for train_index, test_index in skf.split(X,y_binary):
        # get the indexes of the observations assigned for each partition
        X_train, X_val = X.iloc[train_index], X.iloc[test_index]
        y_train, y_val = y_binary.iloc[train_index], y_binary.iloc[test_index]
    
       
        # start counting time
        begin = time.perf_counter()
        # fit the model to the data
        model.fit(X_train, y_train)
        predict_train = model.predict(X_train)
        predict_val = model.predict(X_val)
        # finish counting time
        end = time.perf_counter()
        # check the mean accuracy for the train
        value_train = f1_score(y_train, predict_train)
        accuracy_train = model.score(X_train, y_train)
        # check the mean accuracy for the test
        value_test = f1_score(y_val, predict_val)
        accuracy_test = model.score(X_val, y_val)
        # append the accuracies, the time and the number of iterations in the corresponding list
        score_train.append(value_train)
        score_test.append(value_test)

        acc_train.append(accuracy_train)
        acc_test.append(accuracy_test)
        
        timer.append(end-begin)
        
    # calculate the average and the std for each measure (accuracy, time and number of iterations)
    avg_time = round(np.mean(timer),3)
    avg_train = round(np.mean(score_train),3)
    avg_test = round(np.mean(score_test),3)

    avg_train_a = round(np.mean(acc_train),3)
    avg_test_a = round(np.mean(acc_test),3)

    std_time = round(np.std(timer),2)
    std_train = round(np.std(score_train),2)
    std_test = round(np.std(score_test),2)

    std_train_a = round(np.std(acc_train),2)
    std_test_a = round(np.std(acc_test),2)
    
    return str(avg_time) + '+/-' + str(std_time), str(avg_train) + '+/-' + str(std_train),\
str(avg_test) + '+/-' + str(std_test),str(avg_train_a) + '+/-' + str(std_train_a),\
str(avg_test_a) + '+/-' + str(std_test_a)

def show_results_ns_mv(df, *args):
    """
    Receive an empty dataframe and the different models and call the function avg_score
    """
    count = 0
    # for each model passed as argument
    #scale = scale
    for arg in args:
        # obtain the results provided by avg_score
        time, avg_train, avg_test ,avg_train_a, avg_test_a= avg_score_ns_mv(arg)
        # store the results in the right row
        df.iloc[count] = time, avg_train, avg_test, avg_train_a, avg_test_a
        count+=1
    return df

#### With no missing values and MinMaxScaler

In [189]:
df = pd.DataFrame(columns = ['Time','Train','Test'], index = ['nn','nb',])
show_results2(df, 'minmax',nn,nb)

Time       Train          Test
nn  3.085+/-0.69  0.24+/-0.0  0.239+/-0.01
nb   0.121+/-0.0  0.22+/-0.0   0.22+/-0.01

#### With no missing values and MMS (-1/1)

In [190]:
df = pd.DataFrame(columns = ['Time','Train','Test'], index = ['nn','nb'])
show_results2(df, 'minmax2',nn,nb)

Time         Train          Test
nn  5.671+/-1.28  0.229+/-0.01  0.225+/-0.02
nb   0.126+/-0.0   0.205+/-0.0   0.204+/-0.0

#### With no missing values and no scaling

In [191]:
df = pd.DataFrame(columns = ['Time','Train','Test'], index = ['lr', 'hgb','sgd','rf'])
show_results_ns(df,lr, hgb, sgd,rf)

Time        Train          Test
lr   0.692+/-0.07  0.272+/-0.0   0.27+/-0.01
hgb  1.378+/-0.41  0.313+/-0.0  0.284+/-0.01
sgd  1.613+/-0.16  0.271+/-0.0   0.27+/-0.01
rf   2.622+/-0.07  0.292+/-0.0  0.277+/-0.01

#### Missing values and no scaling

In [192]:
df = pd.DataFrame(columns = ['Time','f1_Train','f1_Test', 'Acc_train', 'Acc_test'], index = [ 'hgb'])
show_results_ns_mv(df, hgb)

Time     f1_Train       f1_Test    Acc_train      Acc_test
hgb  1.022+/-0.1  0.314+/-0.0  0.287+/-0.01  0.774+/-0.0  0.765+/-0.01

lr, hgb - > no scalling is better \
nn, nb -> need scaling (robust is not one)

In [193]:
#Setting the range for class weights
weights = np.linspace(0.0,0.99,200)

#Creating a dictionary grid for grid search
#param_grid = {'class_weight': [{0:x, 1:1.0-x} for x in weights],
             # 'learning_rate': (0.01, 0.1, 1, 10),
             # 'max_leaf_nodes': (3, 10, 30),
             # 'max_depth': (10,30,50)
             # }

#Fitting grid search to the train data with 5 folds
#gridsearch = GridSearchCV(estimator= hgb, 
                          #param_grid= param_grid,
                          #cv=StratifiedKFold(), 
                          #n_jobs=-1, 
                          #scoring='f1', 
                          #verbose=2).fit(X_val_b, y_val_b)

#Ploting the score for different values of weight
#sns.set_style('whitegrid')
#plt.figure(figsize=(12,8))
#weigh_data = pd.DataFrame({ 'score': gridsearch.cv_results_['mean_test_score'], 'weight': (1- weights)})
#sns.lineplot(weigh_data['weight'], weigh_data['score'])
#plt.xlabel('Weight for class 1')
#plt.ylabel('F1 score')
#plt.xticks([round(i/10,1) for i in range(0,11,1)])
#plt.title('Scoring for different class weights', fontsize=24)
#print(gridsearch.best_params_, gridsearch.best_score_, gridsearch.best_estimator_)

In [194]:
df_1.to_csv('DATASET_NM.csv',index_label='encounter_id')
data.to_csv('DATASET.csv', index_label='encounter_id')